In [ ]:
from math import sqrt
import numpy as np
from numpy import concatenate
from matplotlib import pyplot
import pandas as pd
from sklearn.metrics import mean_squared_error
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU, Dropout
import plotly.offline as py
import plotly.graph_objs as graph
from sklearn.preprocessing import MinMaxScaler

# Load the first CSV file (with "Timestamp")
dataset = pd.read_csv("C:/Users/bhave/Desktop/Bitcoin.csv")

# Data Preprocessing
dataset['Weighted_Price'].replace(0, np.nan, inplace=True)
dataset['Weighted_Price'].fillna(method='ffill', inplace=True)

# Data Visualization
btc = graph.Scatter(x=dataset['Timestamp'], y=dataset['Weighted_Price'], name='Price')
py.plot([btc])

# Data Normalization
values = dataset['Weighted_Price'].values.reshape(-1, 1)
values = values.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
values = scaler.fit_transform(values)

# Train-Test Split
train_size = int(len(values) * 0.7)
test_size = len(values) - train_size
train, test = values[0:train_size, :], values[train_size:len(values), :]

look_back = 1

# Data Preparation for LSTM and GRU
def prepare_data(dataset, look_back=1):
    data_x, data_y = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back), 0]
        data_x.append(a)
        data_y.append(dataset[i + look_back, 0])
    return np.array(data_x), np.array(data_y)

train_x, train_y = prepare_data(train, look_back)
test_x, test_y = prepare_data(test, look_back)
train_x = np.reshape(train_x, (train_x.shape[0], 1, train_x.shape[1]))
test_x = np.reshape(test_x, (test_x.shape[0], 1, test_x.shape[1]))

# LSTM Model
model_lstm = Sequential()
model_lstm.add(LSTM(100, input_shape=(train_x.shape[1], train_x.shape[2])))
model_lstm.add(Dense(1))
model_lstm.compile(loss='mae', optimizer='adam')

# Model Training
metadata_lstm = model_lstm.fit(train_x, train_y, epochs=300, batch_size=100, validation_data=(test_x, test_y), verbose=0, shuffle=False)

# Plot Training Loss
pyplot.plot(metadata_lstm.history['loss'], label='train')
pyplot.plot(metadata_lstm.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

# Model Evaluation
yhat = model_lstm.predict(test_x)
pyplot.plot(yhat, label='predicted')
pyplot.plot(test_y, label='true')
pyplot.legend()
pyplot.show()

# Calculate RMSE
yhat_inverse = scaler.inverse_transform(yhat.reshape(-1, 1))
test_y_inverse = scaler.inverse_transform(test_y.reshape(-1, 1))
lstm_rmse = sqrt(mean_squared_error(test_y_inverse, yhat_inverse))
print('LSTM RMSE: %.3f' % lstm_rmse)

# Plot Predicted vs. Actual Prices
pyplot.plot(yhat_inverse, label='predicted')
pyplot.plot(test_y_inverse, label='actual', alpha=0.5)
pyplot.legend()
pyplot.show()

# Create a chart for actual and predicted prices
dates = dataset.tail(len(test_x)).Timestamp
test_y_reshape = test_y_inverse.reshape(len(test_y_inverse))
yhat_reshape = yhat_inverse.reshape(len(yhat_inverse))
actual_chart = graph.Scatter(x=dates, y=test_y_reshape, name='Actual Price')
predict_chart = graph.Scatter(x=dates, y=yhat_reshape, name='Predict Price')
py.plot([predict_chart, actual_chart])

#Roor Mean Squared Error (RMSE)
from math import sqrt

# Assuming you have actual and predicted values
actual_values = test_y_inverse
predicted_values = yhat_inverse

# Calculate RMSE
rmse = sqrt(mean_squared_error(actual_values, predicted_values))

print('RMSE: %.3f' % rmse)


#Mean Absolute Error(MAE)
from sklearn.metrics import mean_absolute_error

# Calculate MAE
mae = mean_absolute_error(actual_values, predicted_values)

print('MAE: %.3f' % mae)

#Mean Absolute Percentage Error(MAPE)
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Calculate MAPE
mape = mean_absolute_percentage_error(actual_values, predicted_values)

print('MAPE: %.3f' % mape)


#Coefficient Of Determination(R-Squared)
from sklearn.metrics import r2_score

# Calculate R-squared
r_squared = r2_score(actual_values, predicted_values)

print('R-squared: %.3f' % r_squared)


